# Stock Market Analysis of Apple, Microsoft, Amazon and Google Stocks In the Past 6 Months


Lets import yfinance library to collect stock market data and other libraries for analysis.

In [1]:
!pip install yfinance
import pandas as pd
import yfinance as yf
import plotly.express as px
from datetime import datetime

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/1b/0f/77716aa9dd84bb1aa5e93c87122af1de89697b6231f6d01d58d4e7c03c14/yfinance-0.2.36-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.7 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.2 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.0-py3-none-any.whl size=135721 sha256=4ecfb0f61e4ee6d0a88a089d00fc53dcf111e080d1582e1a08d5dbdcc1d9683a
  Stored in directory: /root/.cache/pip/wheels/c7/70/ad/212867e96e7004265a69c4aa5dcff00a95f547a67ba26e7e76
Successfully built peewee


* Defining start date and end date for our analysis from the last 6 months till present date.

In [2]:
start_date = datetime.now() - pd.DateOffset(months=6)
end_date = datetime.now()

Defining the 'Tickers', so that we start our analysis for the mentioned Companies' Stocks. 

In [3]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL']

* Looping through each stocks, then pulling the data, and then append the data into a list.

In [4]:
df_list = []
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)
df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])

df = df.reset_index()

print(df.head(100))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

   Ticker       Date        Open        High         Low       Close  \
0    AAPL 2023-07-25  193.330002  194.440002  192.919998  193.619995   
1    AAPL 2023-07-26  193.669998  195.639999  193.320007  194.500000   
2    AAPL 2023-07-27  196.020004  197.199997  192.550003  193.220001   
3    AAPL 2023-07-28  194.669998  196.630005  194.139999  195.830002   
4    AAPL 2023-07-31  196.059998  196.490005  195.259995  196.449997   
..    ...        ...         ...         ...         ...         ...   
95   AAPL 2023-12-07  193.630005  195.000000  193.589996  194.270004   
96   AAPL 2023-12-08  194.199997  195.990005  193.669998  195.710007   
97   AAPL 2023-12-11  193.110001  193.490005  191.419998  193.179993   
98   AAPL 2023-12-12  193.080002  194.720001  191.720001  194.710007   
99   AAPL 2023-12-13  195.089996  198.000000  194.850006  197.960007   

     Adj Close    Volume  
0   193.104477  37283200  
1   193.982147  47471900  
2   192.705551  47460200  
3   195.308609  48291400  


> Requirements:
* Compare the stock performances (Close Metrics) for the 4 mentioned stocks over the past 6 months.
* Compare moving average using rolling window with 10 and 30 steps. (We will use rolling indiators as it is widely used for technical analysis and for marketing analysis)
* Visualize the volatility of all company stocks.
* Analyze the correlation between the stock prices of Apple and Google.

* Compare stock performance (metric: close) for 4 stocks over the past 6 months.

In [5]:
fig = px.line(df, x= 'Date',
              y= 'Close',
              color= 'Ticker',
              title= "Stock Market Performance for the last 6 months")
fig.show()

In [6]:
fig = px.area(df, x='Date', y='Close',
             facet_col = 'Ticker',
             labels = {'Date':'Date','Close':'Closing Price', 'Ticker':'Company'},
             title ='Stock Prices for Apple, Microsoft, Amazon, and Google')
fig.show()

* Compare moving average using rolling window with 10 and 30 steps.

In [7]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop = True)
df['MA30'] = df.groupby('Ticker')['Close'].rolling(window=30).mean().reset_index(0, drop = True)

In [8]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA30'],
                 title=f"{ticker} Moving Averages")
    fig.show()

* Visulaize the volatility of all company stocks

In [9]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop = True)
fig = px.line(df, x='Date', y='Volatility',
             color='Ticker',
             title = 'Volatility of All Companies')
fig.show()

* Analyze the correlation between the stck prices of Apple and Google

In [10]:
# create a DataFrame with the stock prices of Apple and Google
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
google = df.loc[df['Ticker'] == 'GOOGL', ['Date', 'Close']].rename(columns={'Close': 'GOOGL'})
df_corr = pd.merge(apple, google, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='GOOGL',
                 trendline='ols',
                 title='Correlation between Apple and Google')
fig.show()